## Import Libraries

In [1]:
import numpy as np
from matplotlib import image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
from random import randrange
import glob


## Load Images

In [28]:
ZOOM_SIZE = 1
SIGMA = 1

In [29]:
def random_crop(img, width):
    x, y = img.size
    matrix = width

    x1 = randrange(0, x - matrix)
    y1 = randrange(0, y - matrix)
    return img.crop((x1, y1, x1 + matrix, y1 + matrix))

In [30]:
def load_images(folderpath, limit, img_size_crop):
    image_paths = sorted(glob.glob(folderpath))
    img_arr = []
    for img_path in image_paths[:limit]:
        print('Loading', img_path)
        image = Image.open(img_path).convert('L')
        image = random_crop(image, img_size_crop)
        image = np.asarray(image)
        # summarize shape of the pixel array
        img_arr.append(image)
    
    return img_arr

In [31]:
hr_img_arr = load_images("../dataset/DIV2K_valid_HR/*.png", 10, 10)

Loading ../dataset/DIV2K_valid_HR/0801.png
Loading ../dataset/DIV2K_valid_HR/0802.png
Loading ../dataset/DIV2K_valid_HR/0803.png
Loading ../dataset/DIV2K_valid_HR/0804.png
Loading ../dataset/DIV2K_valid_HR/0805.png
Loading ../dataset/DIV2K_valid_HR/0806.png
Loading ../dataset/DIV2K_valid_HR/0807.png
Loading ../dataset/DIV2K_valid_HR/0808.png
Loading ../dataset/DIV2K_valid_HR/0809.png
Loading ../dataset/DIV2K_valid_HR/0810.png


In [32]:
lr_img_arr = load_images("../dataset/DIV2K_valid_LR_bicubic/x2/*.png", 10, 10)

Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0801x2.png
Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0802x2.png
Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0803x2.png
Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0804x2.png
Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0805x2.png
Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0806x2.png
Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0807x2.png
Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0808x2.png
Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0809x2.png
Loading ../dataset/DIV2K_valid_LR_bicubic/x2/0810x2.png


In [33]:
lr_img_arr = [e[::ZOOM_SIZE].flatten().reshape(1, -1) for e in lr_img_arr]

In [34]:
hr_img_arr[0].shape

(10, 10)

In [35]:
lr_img_arr[0].shape

(1, 100)

In [36]:
len(lr_img_arr)

10

## Downsample and Blur HR Images

In [37]:
def matlab_style_gauss2D(shape=(3,3),sigma=0.5):
    """
    2D gaussian mask - should give the same result as MATLAB's
    fspecial('gaussian',[shape],[sigma])
    """
    m,n = [(ss-1.)/2. for ss in shape]
    y,x = np.ogrid[-m:m+1,-n:n+1]
    h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
    h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
    sumh = h.sum()
    if sumh != 0:
        h /= sumh
    return h

In [38]:
import sys
sys.path.append('..')

In [39]:
from python_pipeline.obs_for_SR import obs_for_SR

Run downsampling + blur on all images

In [40]:
transformed_lr_img_arr = np.array([
    obs_for_SR(
        image, 
        matlab_style_gauss2D((7,7), 2),
        SIGMA, 
        ZOOM_SIZE
    ).flatten().reshape(1, -1) for image in hr_img_arr
])

In [41]:
transformed_lr_img_arr[0].shape

(1, 100)

In [42]:
transformed_lr_img_arr.shape

(10, 1, 100)

## Multivariate Gaussian Distribution

In [43]:
from scipy.stats import multivariate_normal as mvn

𝜇̂ = 1/𝑚 ∑𝐱(𝐢) = 𝐱¯<br>
Σ̂ =1/𝑚∑(𝐱(𝐢)−𝜇̂ ) (𝐱(𝐢)−𝜇̂ )𝑇

In [76]:
transformed_lr_img_arr.reshape(10, -1).shape

(10, 100)

In [91]:
mu = np.mean(transformed_lr_img_arr, axis=0).reshape(-1, 1)
cov = np.mean([np.dot((lr_img - mu), (lr_img - mu).T) for lr_img in transformed_lr_img_arr], axis=0)
print(mu.shape, cov.shape)

(100, 1) (100, 100)


In [92]:
cov2.shape

(100, 100)

Calculate for sample image

In [95]:
mvn.pdf(lr_img_arr[0], mean=mu.reshape(-1), cov=cov, allow_singular=True)

1.0779787013733026e-07